In [15]:
import torch
import torchmetrics
from torch.autograd import Variable
from config import Config
from utils import generate_data_loader, data_factory, shuffle, norminy, generate_valid_test_loader
from model import MS_MDA
import numpy as np
from utils import EarlyStopping
import math
import torch.nn.functional as F
import pandas as pd
import os

In [2]:
'''
Set the configuration of the training
'''
config = Config()

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

np.random.seed(100)

metrics = torchmetrics.Accuracy().to(device)

In [3]:
def train_model(source_comb, source_label_comb, target_comb, target_label_comb, target, sess):
    '''
    The main structure of the training framework
    '''
    target_loader, test_loader = generate_valid_test_loader(target_comb, target_label_comb, config) # Split the target data into a valid set and a test set. The model is unable to see the test set during the training
    # target_loader = generate_data_loader(target_comb, target_label_comb, config)
    source_loader = []
    
    # construct multi-source domain data
    for source in range(len(source_comb)):
        source_loader.append(generate_data_loader(source_comb[source], source_label_comb[source], config))

    model = MS_MDA(target_comb.shape[-1], config.num_classes, len(source_loader), config, device)
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate,
                                  weight_decay=config.weight_decay)
    target_iter = iter(target_loader)
    source_iter_list = []
    for i in range(len(source_loader)):
        source_iter_list.append(iter(source_loader[i]))

    real_epoch = 1

    early_stopping = EarlyStopping()

    final_loss_list = []
    final_loss_s_list = []
    final_loss_t_list = []

    for i in range(len(source_loader)):
        final_loss_list.append([])
        final_loss_s_list.append([])
        final_loss_t_list.append([])

    csv_acc_list = []
    csv_loss_list = []

    total_loss_list = []
    total_train_loss_s_list = []
    total_train_loss_t_list = []

    for i in range(len(source_loader)):
        total_train_loss_s_list.append([])
        total_train_loss_t_list.append([])
        total_loss_list.append([])

    cen_list = []
    # Applying the clustering centroid varaibles
    for mark in range(len(source_loader)):
        cen_temp = Variable(torch.FloatTensor(config.num_classes, 64)).fill_(0).to(device)
        cen_temp.requires_grad_(True)
        cen_list.append(cen_temp)
    
    # Calculate the initial clustering centroid of the target domain and each source domain
    for mark in range(len(source_loader)):
        source_centroid, target_centroid = model.calculate_centroid(source_loader[mark], target_loader, mark)
        cen_list[mark].data = source_centroid.data.clone()

    for epoch in range(config.epochs):
        model.train()

        for mark in range(len(source_loader)):

            try:
                source_data_batch, source_label_batch, source_weight_batch, _ = next(source_iter_list[mark])
            except Exception as err:
                source_iter_list[mark] = iter(source_loader[mark])
                source_data_batch, source_label_batch, source_weight_batch, _ = next(source_iter_list[mark])

            try:
                target_data_batch, target_label_batch, _, _ = next(target_iter)
            except Exception as err:
                target_iter = iter(target_loader)
                target_data_batch, target_label_batch, _, _ = next(target_iter)

            source_data_batch = source_data_batch.to(device).float()
            source_label_batch = source_label_batch.to(device).long()

            target_data_batch = target_data_batch.to(device).float()

            c_loss, t_loss, c_loss_cluster, t_loss_cluster = model(source_data_batch, source_label_batch, source_weight_batch,
                                           target_data_batch, mark, cen_list[mark])

            gamma = 2 / (1 + math.exp(-10 * epoch / config.epochs)) - 1

            loss = c_loss + gamma*t_loss + gamma*c_loss_cluster + gamma * t_loss_cluster

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss_list[mark].append(loss.item())
            total_train_loss_s_list[mark].append(c_loss.item())
            total_train_loss_t_list[mark].append(t_loss.item())

        if math.floor(((epoch + 1) * config.batch_size) / config.data_length) == real_epoch:

            model.eval()

            for m in range(len(source_loader)):
                source_centroid, target_centroid = model.calculate_centroid(source_loader[m], target_loader, m)
                centroid = Variable(torch.FloatTensor(config.num_classes, source_centroid.shape[-1])).fill_(0).to(device)
                # Avoid the empty set during the clustering
                for c in range(centroid.shape[0]):
                    non_zero = torch.nonzero(target_centroid[c, :])
                    if min(non_zero.shape) == 0:
                        print('Centroid Number is not full')
                        centroid[c, :] = source_centroid[c, :].data.clone()
                    else:
                        centroid[c, :] = ((source_centroid[c, :] + target_centroid[c, :])/2).data.clone()
                cen_list[m].data = centroid.data.clone()

            for mark in range(len(source_loader)):
                print('Train source ' + str(
                    mark + 1) + ' Total loss: {:.4f} - source loss: {:.4f} - target_loss: {:.4f}'.
                      format(np.mean(total_loss_list[mark]), np.mean(total_train_loss_s_list[mark]),
                             np.mean(total_train_loss_t_list[mark])))

                final_loss_list[mark].append(np.mean(total_loss_list[mark]))
                final_loss_s_list[mark].append(np.mean(total_train_loss_s_list[mark]))
                final_loss_t_list[mark].append(np.mean(total_train_loss_t_list[mark]))

                total_loss_list[mark] = []
                total_train_loss_s_list[mark] = []
                total_train_loss_t_list[mark] = []

            real_epoch += 1
            acc_list = []
            loss_list = []
            # evaluate the performance of the model on the test set
            for t_data, t_label, _, _ in test_loader:
                t_data = t_data.to(device).float()
                t_label = t_label.to(device).long()
                t_pred = model.predict_class(t_data)
                for i in range(len(t_pred)):
                    t_pred[i] = F.softmax(t_pred[i], dim=-1)
                pred = sum(t_pred) / len(t_pred)
                cls_loss = F.nll_loss(F.log_softmax(pred, dim=1), t_label)
                acc = metrics(torch.argmax(pred, dim=-1), t_label)
                acc_list.append(acc.item())
                loss_list.append(cls_loss.item())
            print('Epoch {:.0f}: Testing Acc: {:.4f} - Testing Loss: {:.4f}'.format(real_epoch - 1, np.mean(acc_list),
                                                                                    np.mean(loss_list)))
            csv_acc_list.append(np.mean(acc_list))
            csv_loss_list.append(np.mean(loss_list))

            early_stopping(np.mean(acc_list), model)

    print('Best score recorded: ', early_stopping.best_score)
    csv_dir = os.path.join(config.result_path, str(sess), str(target), 'csv')
    model_dir = os.path.join(config.result_path, str(sess), str(target), 'model')
    if not os.path.exists(csv_dir):
        os.makedirs(csv_dir)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    torch.save(early_stopping.save_model.state_dict(), os.path.join(model_dir, 'model.pth'))

    writer = pd.ExcelWriter(os.path.join(csv_dir, 'result.xls'), engine='openpyxl')
    csv_dict = {
        'Test Accuracy': csv_acc_list,
        'Test Loss': csv_loss_list
    }
    df_test = pd.DataFrame(csv_dict)
    df_test.to_excel(writer, sheet_name='test result')
    for mark in range(len(source_loader)):
        data_dict = {
            'Train Loss Total': final_loss_list[mark],
            'Train Loss Source': final_loss_s_list[mark],
            'Train Loss Target': final_loss_t_list[mark],
        }
        df = pd.DataFrame(data_dict)
        df.to_excel(writer, sheet_name='Source' + str(mark))
    writer.save()

In [4]:
def eval(source_comb, source_label_comb, target_comb, target_label_comb, target, sess):
    '''
    Test the model after the training
    '''
    target_loader, test_loader = generate_valid_test_loader(target_comb, target_label_comb, config)
    # target_loader = generate_data_loader(target_comb, target_label_comb, config)
    source_loader = []
    for source in range(len(source_comb)):
        source_loader.append(generate_data_loader(source_comb[source], source_label_comb[source], config))

    model = MS_MDA(target_comb.shape[-1], config.num_classes, len(source_loader), config, device)
    model.to(device)
    model_dir = os.path.join(config.result_path, str(sess), str(target), 'model', 'model.pth')
    state_dict = torch.load(model_dir, map_location=device)
    model.load_state_dict(state_dict)

    model.eval()

    acc_list = []
    loss_list = []
    # Evaluate the performance on the test set
    for t_data, t_label, _, _ in test_loader:
        t_data = t_data.to(device).float()
        t_label = t_label.to(device).long()
        t_pred = model.predict_class(t_data)
        for i in range(len(t_pred)):
            t_pred[i] = F.softmax(t_pred[i], dim=-1)
        pred = sum(t_pred) / len(t_pred)
        cls_loss = F.nll_loss(F.log_softmax(pred, dim=1), t_label)
        acc = metrics(torch.argmax(pred, dim=-1), t_label)
        acc_list.append(acc.item())
        loss_list.append(cls_loss.item())

    print('Testing Acc: {:.4f} - Testing Loss: {:.4f}'.format(np.mean(acc_list), np.mean(loss_list)))

In [5]:
def train_cross_subject(sess):
    '''
    Code for implementing cross-subject experiments
    '''
    data_package, label_package = data_factory(config.file_path, config)
    data_sess, label_sess = data_package[str(sess)], label_package[str(sess)]

    total_pred_list = []
    total_real_list = []

    for target in range(config.source_number):
        print('Target ', target)
        target_comb, target_label_comb = data_sess[target], label_sess[target]
        target_comb = norminy(target_comb)
        target_comb, target_label_comb = shuffle(target_comb, target_label_comb)

        source_comb, source_label_comb = [], []
        for source in range(config.source_number):
            if source != target:
                source_comb_temp, source_label_comb_temp = data_sess[source], label_sess[source]
                source_comb_temp = norminy(source_comb_temp)
                source_comb_temp, source_label_comb_temp = shuffle(source_comb_temp, source_label_comb_temp)
                source_comb.append(source_comb_temp)
                source_label_comb.append(source_label_comb_temp)

        train_model(source_comb, source_label_comb, target_comb, target_label_comb, target, sess)
        # eval(source_comb, source_label_comb, target_comb, target_label_comb, target, sess)

In [6]:
def train_cross_session(sub):
    '''
    Code for implementing cross-session experiments
    '''
    data_package, label_package = data_factory(config.file_path, config)

    total_pred_list = []
    total_real_list = []

    for target in range(1, 4):
        target_comb, target_label_comb = data_package[str(target)][sub], label_package[str(target)][sub]
        target_comb = norminy(target_comb)
        target_comb, target_label_comb = shuffle(target_comb, target_label_comb)

        source_comb, source_label_comb = [], []
        for source in range(1, 4):
            if source != target:
                source_comb_temp, source_label_comb_temp = data_package[str(source)][sub], label_package[str(source)][sub]
                source_comb_temp = norminy(source_comb_temp)
                source_comb_temp, source_label_comb_temp = shuffle(source_comb_temp, source_label_comb_temp)
                source_comb.append(source_comb_temp)
                source_label_comb.append(source_label_comb_temp)

        train_model(source_comb, source_label_comb, target_comb, target_label_comb, target, sub)
        # eval(source_comb, source_label_comb, target_comb, target_label_comb, target, sub)

In [7]:
# for sess in range(1, 4):  # cross-subject
#     train_cross_subject(sess)
for sub in range(15):   # cross-session
    train_cross_session(sub)

Processing Session  1
Processing Session  2
Processing Session  3
Processing Session  label.mat
Train source 1 Total loss: 0.4078 - source loss: 0.3373 - target_loss: 0.7375
Train source 2 Total loss: 0.3888 - source loss: 0.3188 - target_loss: 0.7253
Epoch 1: Testing Acc: 0.7891 - Testing Loss: 0.8188
Train source 1 Total loss: 0.1077 - source loss: 0.0464 - target_loss: 0.2738
Train source 2 Total loss: 0.1013 - source loss: 0.0387 - target_loss: 0.2462
Epoch 2: Testing Acc: 0.8719 - Testing Loss: 0.6998
Train source 1 Total loss: 0.1265 - source loss: 0.0598 - target_loss: 0.1275
Train source 2 Total loss: 0.1148 - source loss: 0.0521 - target_loss: 0.1110
Epoch 3: Testing Acc: 0.8938 - Testing Loss: 0.6837
Train source 1 Total loss: 0.0992 - source loss: 0.0207 - target_loss: 0.0824
Train source 2 Total loss: 0.0953 - source loss: 0.0220 - target_loss: 0.0696
Epoch 4: Testing Acc: 0.8625 - Testing Loss: 0.6865
Train source 1 Total loss: 0.0895 - source loss: 0.0095 - target_loss: 0

Train source 1 Total loss: 0.2156 - source loss: 0.0032 - target_loss: 0.0029
Train source 2 Total loss: 0.1779 - source loss: 0.0032 - target_loss: 0.0017
Epoch 40: Testing Acc: 0.8844 - Testing Loss: 0.6672
Train source 1 Total loss: 0.2277 - source loss: 0.0135 - target_loss: 0.0030
Train source 2 Total loss: 0.3157 - source loss: 0.0989 - target_loss: 0.0021
Epoch 41: Testing Acc: 0.8859 - Testing Loss: 0.6656
Train source 1 Total loss: 0.2174 - source loss: 0.0018 - target_loss: 0.0034
Train source 2 Total loss: 0.1829 - source loss: 0.0103 - target_loss: 0.0016
Epoch 42: Testing Acc: 0.8828 - Testing Loss: 0.6687
Train source 1 Total loss: 0.3505 - source loss: 0.0960 - target_loss: 0.0030
Train source 2 Total loss: 0.1790 - source loss: 0.0048 - target_loss: 0.0016
Epoch 43: Testing Acc: 0.8828 - Testing Loss: 0.6687
Train source 1 Total loss: 0.2167 - source loss: 0.0022 - target_loss: 0.0033
Train source 2 Total loss: 0.2777 - source loss: 0.0872 - target_loss: 0.0019
Epoch 44

Train source 1 Total loss: 0.2149 - source loss: 0.0043 - target_loss: 0.0061
Train source 2 Total loss: 0.1906 - source loss: 0.0067 - target_loss: 0.0022
Epoch 80: Testing Acc: 0.8844 - Testing Loss: 0.6672
Train source 1 Total loss: 0.2051 - source loss: 0.0017 - target_loss: 0.0059
Train source 2 Total loss: 0.1782 - source loss: 0.0124 - target_loss: 0.0016
Epoch 81: Testing Acc: 0.8828 - Testing Loss: 0.6688
Train source 1 Total loss: 0.2400 - source loss: 0.0135 - target_loss: 0.0050
Train source 2 Total loss: 0.1843 - source loss: 0.0092 - target_loss: 0.0016
Epoch 82: Testing Acc: 0.8906 - Testing Loss: 0.6609
Train source 1 Total loss: 0.3032 - source loss: 0.0699 - target_loss: 0.0054
Train source 2 Total loss: 0.1837 - source loss: 0.0039 - target_loss: 0.0015
Epoch 83: Testing Acc: 0.8859 - Testing Loss: 0.6656
Train source 1 Total loss: 0.2025 - source loss: 0.0053 - target_loss: 0.0042
Train source 2 Total loss: 0.1894 - source loss: 0.0146 - target_loss: 0.0013
Epoch 84

Train source 1 Total loss: 0.3474 - source loss: 0.0811 - target_loss: 0.0147
Train source 2 Total loss: 0.1855 - source loss: 0.0170 - target_loss: 0.0069
Epoch 20: Testing Acc: 0.7578 - Testing Loss: 0.7943
Train source 1 Total loss: 0.3389 - source loss: 0.0789 - target_loss: 0.0132
Train source 2 Total loss: 0.2773 - source loss: 0.0852 - target_loss: 0.0062
Epoch 21: Testing Acc: 0.7406 - Testing Loss: 0.8108
Train source 1 Total loss: 0.2406 - source loss: 0.0042 - target_loss: 0.0115
Train source 2 Total loss: 0.1746 - source loss: 0.0056 - target_loss: 0.0046
Epoch 22: Testing Acc: 0.7406 - Testing Loss: 0.8109
Train source 1 Total loss: 0.3269 - source loss: 0.0662 - target_loss: 0.0107
Train source 2 Total loss: 0.1889 - source loss: 0.0146 - target_loss: 0.0040
Epoch 23: Testing Acc: 0.7484 - Testing Loss: 0.8031
Train source 1 Total loss: 0.2426 - source loss: 0.0037 - target_loss: 0.0096
Train source 2 Total loss: 0.1755 - source loss: 0.0027 - target_loss: 0.0026
Epoch 24

Train source 1 Total loss: 0.2762 - source loss: 0.0109 - target_loss: 0.0046
Train source 2 Total loss: 0.2328 - source loss: 0.0066 - target_loss: 0.0063
Epoch 60: Testing Acc: 0.7750 - Testing Loss: 0.7765
Train source 1 Total loss: 0.3253 - source loss: 0.0120 - target_loss: 0.0090
Train source 2 Total loss: 0.2204 - source loss: 0.0088 - target_loss: 0.0048
Epoch 61: Testing Acc: 0.7844 - Testing Loss: 0.7657
Train source 1 Total loss: 0.4759 - source loss: 0.0903 - target_loss: 0.0121
Train source 2 Total loss: 0.2398 - source loss: 0.0188 - target_loss: 0.0043
Epoch 62: Testing Acc: 0.7641 - Testing Loss: 0.7869
Train source 1 Total loss: 0.3268 - source loss: 0.0113 - target_loss: 0.0125
Train source 2 Total loss: 0.2975 - source loss: 0.0714 - target_loss: 0.0063
Epoch 63: Testing Acc: 0.7406 - Testing Loss: 0.8110
Train source 1 Total loss: 0.4082 - source loss: 0.0902 - target_loss: 0.0098
Train source 2 Total loss: 0.2808 - source loss: 0.0609 - target_loss: 0.0066
Epoch 64

...

Train source 1 Total loss: 0.2277 - source loss: 0.0169 - target_loss: 0.0086
Train source 2 Total loss: 0.5290 - source loss: 0.1218 - target_loss: 0.0085
Epoch 40: Testing Acc: 0.7594 - Testing Loss: 0.7615
Train source 1 Total loss: 0.2382 - source loss: 0.0122 - target_loss: 0.0098
Train source 2 Total loss: 0.3057 - source loss: 0.0064 - target_loss: 0.0112
Epoch 41: Testing Acc: 0.7734 - Testing Loss: 0.7560
Train source 1 Total loss: 0.3072 - source loss: 0.0743 - target_loss: 0.0081
Train source 2 Total loss: 0.2825 - source loss: 0.0071 - target_loss: 0.0096
Epoch 42: Testing Acc: 0.7531 - Testing Loss: 0.7790
Train source 1 Total loss: 0.2225 - source loss: 0.0040 - target_loss: 0.0098
Train source 2 Total loss: 0.2787 - source loss: 0.0164 - target_loss: 0.0077
Epoch 43: Testing Acc: 0.7359 - Testing Loss: 0.7801
Train source 1 Total loss: 0.3004 - source loss: 0.0826 - target_loss: 0.0059
Train source 2 Total loss: 0.2994 - source loss: 0.0076 - target_loss: 0.0081
Epoch 44

Train source 1 Total loss: 0.2289 - source loss: 0.0146 - target_loss: 0.0059
Train source 2 Total loss: 0.3574 - source loss: 0.0711 - target_loss: 0.0068
Epoch 80: Testing Acc: 0.7562 - Testing Loss: 0.8016
Train source 1 Total loss: 0.3299 - source loss: 0.0965 - target_loss: 0.0043
Train source 2 Total loss: 0.3906 - source loss: 0.0692 - target_loss: 0.0079
Epoch 81: Testing Acc: 0.7422 - Testing Loss: 0.8158
Train source 1 Total loss: 0.3251 - source loss: 0.0841 - target_loss: 0.0050
Train source 2 Total loss: 0.3756 - source loss: 0.0760 - target_loss: 0.0069
Epoch 82: Testing Acc: 0.7281 - Testing Loss: 0.8305
Train source 1 Total loss: 0.2327 - source loss: 0.0118 - target_loss: 0.0060
Train source 2 Total loss: 0.2648 - source loss: 0.0081 - target_loss: 0.0068
Epoch 83: Testing Acc: 0.7172 - Testing Loss: 0.8209
Train source 1 Total loss: 0.2246 - source loss: 0.0034 - target_loss: 0.0044
Train source 2 Total loss: 0.2714 - source loss: 0.0142 - target_loss: 0.0086
Epoch 84

Train source 1 Total loss: 0.3035 - source loss: 0.0933 - target_loss: 0.0070
Train source 2 Total loss: 0.2060 - source loss: 0.0052 - target_loss: 0.0092
Epoch 20: Testing Acc: 0.8812 - Testing Loss: 0.6715
Train source 1 Total loss: 0.2055 - source loss: 0.0134 - target_loss: 0.0071
Train source 2 Total loss: 0.2861 - source loss: 0.0636 - target_loss: 0.0109
Epoch 21: Testing Acc: 0.8609 - Testing Loss: 0.6964
Train source 1 Total loss: 0.2217 - source loss: 0.0155 - target_loss: 0.0067
Train source 2 Total loss: 0.2847 - source loss: 0.0605 - target_loss: 0.0085
Epoch 22: Testing Acc: 0.9000 - Testing Loss: 0.6513
Train source 1 Total loss: 0.2038 - source loss: 0.0057 - target_loss: 0.0063
Train source 2 Total loss: 0.2940 - source loss: 0.0646 - target_loss: 0.0099
Epoch 23: Testing Acc: 0.8219 - Testing Loss: 0.7271
Train source 1 Total loss: 0.3094 - source loss: 0.0716 - target_loss: 0.0077
Train source 2 Total loss: 0.2847 - source loss: 0.0541 - target_loss: 0.0078
Epoch 24

Train source 1 Total loss: 0.3147 - source loss: 0.0928 - target_loss: 0.0038
Train source 2 Total loss: 0.2198 - source loss: 0.0040 - target_loss: 0.0036
Epoch 60: Testing Acc: 0.8812 - Testing Loss: 0.6704
Train source 1 Total loss: 0.2045 - source loss: 0.0068 - target_loss: 0.0040
Train source 2 Total loss: 0.2246 - source loss: 0.0053 - target_loss: 0.0033
Epoch 61: Testing Acc: 0.8906 - Testing Loss: 0.6613
Train source 1 Total loss: 0.2192 - source loss: 0.0065 - target_loss: 0.0055
Train source 2 Total loss: 0.3290 - source loss: 0.0906 - target_loss: 0.0044
Epoch 62: Testing Acc: 0.8906 - Testing Loss: 0.6610
Train source 1 Total loss: 0.2956 - source loss: 0.0731 - target_loss: 0.0038
Train source 2 Total loss: 0.2407 - source loss: 0.0150 - target_loss: 0.0043
Epoch 63: Testing Acc: 0.8625 - Testing Loss: 0.6884
Train source 1 Total loss: 0.2083 - source loss: 0.0019 - target_loss: 0.0036
Train source 2 Total loss: 0.2305 - source loss: 0.0105 - target_loss: 0.0028
Epoch 64

Train source 1 Total loss: 0.1725 - source loss: 0.0043 - target_loss: 0.0019
Train source 2 Total loss: 0.2814 - source loss: 0.0770 - target_loss: 0.0055
Epoch 100: Testing Acc: 0.8734 - Testing Loss: 0.6781
Best score recorded:  0.9234375


In [16]:
def plot_figure_cross_subject(file_path, sess):
    sess_pth = os.path.join(file_path, str(sess))
    acc_list = []
    for sub in range(15):
        data = pd.read_excel(os.path.join(sess_pth, str(sub), 'csv', 'result.xls'), sheet_name='test result').values[:, 1]
        acc_list.append(max(data))

    return acc_list

In [17]:
def plot_figure_cross_session(file_path, sess):
    acc_list = []
    for sub in range(15):
        sub_path = os.path.join(file_path, str(sub))
        data = pd.read_excel(os.path.join(sub_path, str(sess), 'csv', 'result.xls'), sheet_name='test result').values[:, 1]
        acc_list.append(max(data))

    return acc_list

In [18]:
def total_acc(file_path1):
    acc1 = []
    for sess in range(1, 4):
        acc_list = plot_figure_cross_session(file_path1, str(sess))
        acc1.extend(acc_list)
        
    print(np.std(acc1))
    print(np.mean(acc1))

In [19]:
total_acc('result/valid_test/cross_session2/batch_size_64/with_cluster_th_0.9/without_weight')

0.059469405941673534
0.9478125
